In [51]:
import os
import re
import nltk
import string
import numpy as np
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# Question 1: Preprocessing

In [156]:
def extract_data(folder, new_folder):
    files = os.listdir(folder)
    for file in files:
        path = os.path.join(os.getcwd(), folder, file)
        with open(path) as fp:
            soup = BeautifulSoup(fp, 'html.parser')
            text = soup.findAll("text")[0].text
            title = soup.findAll("title")[0].text
            final_text = title + " " + text
        new_file_path = os.path.join(os.getcwd(), new_folder, file)
        with open(new_file_path, "w") as fw:
            fw.write(final_text)
            fw.close()

In [157]:
new_folder = 'Dataset'
try:
    os.mkdir(new_folder)
except:
    print(new_folder, " already exists.")
file_map = extract_data('CSE508_Winter2023_Dataset', new_folder)



Dataset  already exists.


In [158]:
def preprocess(text):

    text = text.lower()

    tokens = word_tokenize(text)
    
    final = [word for word in tokens if word not in stop_words]

    tokens = [word for word in final if word not in string.punctuation]
     
    final = [word for word in tokens if len(re.findall(r'\s+', word)) == 0]
    
    return final

In [159]:
L = []
idtoName = {}
files = os.listdir("Dataset")
for i, file in enumerate(files):
    idtoName[i] = file
    path = os.path.join(os.getcwd(), "Dataset", file)
    with open(path) as fp:
        text = fp.read()
        L.append(preprocess(text))

# Question 2: Boolean Queries

## Unigram Inverted Index

In [160]:
def unigram_inverted_index(doc_list):
    uni_inv_idx = {}
    for doc_id, tokens in enumerate(doc_list):
        for idx, token in enumerate(tokens):
            if token in uni_inv_idx:
                if doc_id not in uni_inv_idx[token]:
                    uni_inv_idx[token].append(doc_id)
            else:
                uni_inv_idx[token] = [doc_id]

    return uni_inv_idx

In [161]:
uni_inv_idx = unigram_inverted_index(L)

<h3>Saving to Pickle

In [162]:
import pickle
filehandler = open("uni_inv_idx.obj","wb")
pickle.dump(uni_inv_idx, filehandler)
filehandler.close()

<h3>Loading from Pickle

In [163]:
file = open("uni_inv_idx.obj",'rb')
uni_inv_idx = pickle.load(file)
file.close()

# Queries

In [164]:
def andQuery(L1, L2):
    ans = []
    comparison = 0
    len1, len2 = len(L1), len(L2)
    
    i , j = 0, 0
    while i < len1 and j < len2:
        if L1[i] == L2[j]:
            ans.append(L1[i])
            i += 1
            j += 1
        elif L1[i] < L2[j]:
            i += 1
        else:
            j += 1
        comparison += 1
    
    return ans, comparison

In [165]:
def andNotQuery(L1, L2):
    _L2 = [i for i in range(1400) if i not in L2]   
    return andQuery(L1, _L2)

In [166]:
def orQuery(L1, L2):
    ans = []
    comparison = 0
    len1, len2 = len(L1), len(L2)
    
    i , j = 0, 0
    while i < len1 and j < len2:
        if L1[i] == L2[j]:
            ans.append(L1[i])
            i += 1
            j += 1
        elif L1[i] < L2[j]:
            ans.append(L1[i])
            i += 1
        else:
            ans.append(L2[j])
            j += 1
            
        comparison += 1
    
    while i < len1:
        ans.append(L1[i])
        i += 1
    
    while j < len2:
        ans.append(L2[j])
        j += 1

    return ans, comparison

In [167]:
def orNotQuery(L1, L2):
    _L2 = [i for i in range(1400) if i not in L2]   
    return orQuery(L1, _L2)

## General Queries

In [168]:
def process_query(i, query, operator, comparisons):
    if i == len(operator):
        return query, comparisons
    
    res = []
    comp = 0

    if 'OR' in operator[i] and 'NOT' not in operator[i]:
        # print('OR')
        res, comp = orQuery(query[0], query[1])

    if 'AND' in operator[i] and 'NOT' not in operator[i]:
        # print('AND')
        res, comp = andQuery(query[0], query[1])

    if 'AND NOT' in operator[i]:
        # print('AND NOT')
        res, comp = andNotQuery(query[0], query[1])
    
    if 'OR NOT' in operator[i]:
        # print('OR NOT')
        res, comp = orNotQuery(query[0], query[1])
    
    del query[ : 2]
    query.insert(0, res)
    comparisons += comp
    
    return process_query(i + 1, query, operator, comparisons)


# Unigram query input and output

In [169]:
def unigram_queries(queries, operand, uni_inv_idx):
    queries_expression, no_of_docs, doc_names, no_of_comp = list(), list(), list(), list()
    for idx in range(len(queries)):
        op = operand[idx]
        ip1 = queries[idx]
        op = op.split(',')
        ip1 = preprocess(ip1)
        query = [uni_inv_idx[i] if i in uni_inv_idx else [] for i in ip1]

        if len(query) != len(op) + 1:
            queries_expression.append("Inappropriate query !!!. Input Mismatch")
            no_of_docs.append(-1)
            doc_names.append(list())
            no_of_comp.append(-1)
            continue

        sent = ""
        p, q = 0, 0
        for idx in range(len(query) + len(op)):
            if idx % 2 == 0:
                sent += ip1[p] + " "
                p += 1
            else:
                sent += op[q] + " "
                q += 1

        comparisons = 0
        output, comparisons = process_query(0, query, op, comparisons)
        docs = [idtoName[i] for i in output[0]]
        queries_expression.append(sent)
        no_of_docs.append(len(output[0]))
        doc_names.append(docs)
        no_of_comp.append(comparisons)
    
    return queries_expression, no_of_docs, doc_names, no_of_comp

    

In [170]:
N = int(input("Enter the number of queries."))
count = 1
queries = []
operand = []
while count <= N:
    query = input("Query : ")
    oper = input("Operand : ")
    queries.append(query)
    operand.append(oper)
    count +=1

queries_expression, no_of_docs, doc_names, no_of_comp = unigram_queries(queries, operand, uni_inv_idx)
for idx in range(len(queries)):
    print(f"Query {idx + 1}: ", queries_expression[idx])
    print(f"Number of documents retrieved for query {idx + 1}: ", no_of_docs[idx])
    print(f"Names of the documents retrieved for query {idx + 1}: ", doc_names[idx])
    print(f"Number of comparisons required for query {idx + 1}: ", no_of_comp[idx])

Query 1:  stream OR constant AND vorticity 
Number of documents retrieved for query 1:  12
Names of the documents retrieved for query 1:  ['cranfield0002', 'cranfield0064', 'cranfield0073', 'cranfield0128', 'cranfield0329', 'cranfield0373', 'cranfield0388', 'cranfield0601', 'cranfield0664', 'cranfield1080', 'cranfield1082', 'cranfield1204']
Number of comparisons required for query 1:  538


# Question 3: Phrase Queries

## Bigram Inverted Index

In [171]:
def bigram_inverted_index(L, files):
    bi_inv_idx = {}
    for doc_id, tokens in enumerate(L):
        for idx, _ in enumerate(tokens):
#             print(tokens[idx])
            if idx <= len(tokens) - 2:
                bigram_word = tokens[idx] + " " + tokens[idx + 1]
                if bigram_word not in bi_inv_idx:
                    bi_inv_idx[bigram_word] = list()
                    bi_inv_idx[bigram_word].append(doc_id)
                else:
                    if doc_id not in bi_inv_idx[bigram_word]:
                        bi_inv_idx[bigram_word].append(doc_id)
                    
    return bi_inv_idx

In [172]:
bi_inv_idx = bigram_inverted_index(L, files)

In [173]:
filehandler = open("bi_inv_idx.obj","wb")
pickle.dump(bi_inv_idx, filehandler)
filehandler.close()

In [174]:
file = open("bi_inv_idx.obj",'rb')
bi_inv_idx = pickle.load(file)
file.close()

## Bigram Queries

In [175]:
def bigram_queries(queries, bi_inv_idx):
    no_of_docs = []
    doc_names = []
    for query in queries:
        query = preprocess(query)
#         print(query)
        bigram_words  = []
        for idx in range(len(query)):
            if idx <= len(query) - 2:
                bigram_words.append(query[idx] + " " + query[idx + 1])
#         print(bigram_words)
        operand = []  
        for idx in range(len(bigram_words) - 1):
            operand.append('AND')

#         print(operand)
        query_doc_list = [bi_inv_idx[i] if i in bi_inv_idx else [] for i in bigram_words]
#         print(query_doc_list)
        
        comparisons = 0
        output, _ = process_query(0, query_doc_list, operand, comparisons)
        docs = [idtoName[i] for i in output[0]]
        no_of_docs.append(len(output[0]))
        doc_names.append(docs)
        
    return no_of_docs, doc_names

        

<h1> Positional Queries

In [176]:
def positional_index(doc_list):
    pos_idx = {}
    for doc_id, tokens in enumerate(doc_list):
        for idx, token in enumerate(tokens):
            if token in pos_idx:
                if doc_id not in pos_idx[token]:
                    pos_idx[token][doc_id] = []
                    pos_idx[token][doc_id].append(idx)
                else:
                    pos_idx[token][doc_id].append(idx)
            else:
                pos_idx[token] = {}
                pos_idx[token][doc_id] = []
                pos_idx[token][doc_id].append(idx)
    return pos_idx

In [177]:
pos_index = positional_index(L)

In [178]:
filehandler = open("pos_idx.obj","wb")
pickle.dump(pos_index, filehandler)
filehandler.close()

In [179]:
file = open("pos_idx.obj",'rb')
pos_index = pickle.load(file)
file.close()

In [193]:
def phrasal_queries(queries,pos_index,file_map):
    phrasal_doc_name = []
    phrasal_doc_len = []

    for query in queries:
        try:
            doc_name = []
            txt = preprocess(query)
            if len(txt)==0:
                continue

            elif len(txt) == 1:
                for doc in pos_index[txt[0]]:
                    doc_name.append(file_map[doc])

            else:
                for doc_id in pos_index[txt[0]].keys():
                    for pos in pos_index[txt[0]][doc_id]:
                        flag = True
                        for i in range(1, len(txt)):
                            if doc_id not in pos_index[txt[i]] or (pos + i) not in pos_index[txt[i]][doc_id]:
                                flag = False
                                break
                        if flag:
                            doc_name.append(file_map[doc_id])
            phrasal_doc_name.append(doc_name)
            phrasal_doc_len.append(len(doc_name))
        except:
            phrasal_doc_name.append([])
            phrasal_doc_len.append(0)

    return phrasal_doc_len, phrasal_doc_name


## Final Query Input and Output

In [202]:
N = int(input("Enter the number of queries."))
count = 1
queries = []
while count <= N:
    query = input("Query : ")
    queries.append(query)
    count +=1

no_of_docs, doc_names = bigram_queries(queries, bi_inv_idx)
phrasal_len, phrasal_names = phrasal_queries(queries, pos_index, idtoName)
for idx in range(len(queries)):
    print(f"Number of documents retrieved for query {idx + 1} using bigram inverted index: ", no_of_docs[idx])
    print(f"Names of documents retrieved for query  {idx + 1} using bigram inverted index: ", doc_names[idx])
    print(f"Number of documents retrieved for query {idx + 1} using positional index: ", phrasal_len[idx])
    print(f"Names of documents retrieved for query  {idx + 1} using positional index: ", phrasal_names[idx])


Number of documents retrieved for query 1 using bigram inverted index:  4
Names of documents retrieved for query  1 using bigram inverted index:  ['cranfield0002', 'cranfield0667', 'cranfield1072', 'cranfield1235']
Number of documents retrieved for query 1 using positional index:  4
Names of documents retrieved for query  1 using positional index:  ['cranfield0002', 'cranfield0667', 'cranfield1072', 'cranfield1235']
